In [ ]:
!pip install opencv-python

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import keras
from sklearn.metrics import f1_score
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
datapath = "dataset/"
img_width, img_height = 128,128
classes = ["manipuri", "bharatanatyam", "odissi", "kathakali", "kathak", "sattriya", "kuchipudi", "mohiniyattam"]
class_names_label = {classes:i for i, classes in enumerate(set(classes))}
num_classes = len(classes)

In [4]:
def load_images(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_width, img_height),cv2.INTER_LINEAR)
    return resized

In [5]:
def norm_images(images):
    images = np.array(images,dtype = 'float32') / 255.  
    mean_img = np.mean(images,axos = 0)
    std_img = np.std(images,axis = 0)
    norm_img = (images - mean_img)/std_img
    return norm_img

In [23]:
def load_dataset(folder_name):
#     folders = ["train", "test"]
#     data = []
#     for dataset in folders:
    images = []
    labels = []
    label_file = pd.read_csv(os.path.join(datapath,folder_name+".csv"))
    for file in tqdm(os.listdir(os.path.join(datapath,folder_name))):
        if(folder_name == "train"):
            label = class_names_label[label_file[label_file["Image"] == file]["target"].values[0]]
#                 label = to_categorical(label)
            labels.append(label)
        else:
            labels.append(file)
        image = load_images(os.path.join(datapath,folder_name,file))
        images.append(image)
    norm_img = np.array(images,dtype = 'float32') / 255.  
    
#         labels = np.array(labels, dtype = 'int16')
#     data.append((norm_img,labels))
    return (norm_img,labels)

In [24]:
(x_train, y_train)=load_dataset("train")

(x_test, img_name) = load_dataset("test")

100%|██████████| 156/156 [00:00<00:00, 208.45it/s]


In [25]:
y_train = np.array(y_train, dtype = 'int16')
y_train = to_categorical(y_train)

print(y_train.shape)

(364, 8)


In [26]:
model = Sequential()
model.add(Convolution2D(32, (3,3), activation='relu', padding='same',input_shape = (128,128,3)))
#if you resize the image above, change the input shape
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 30, 64)       

In [27]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')

In [ ]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.3, callbacks=[early_stops])

Train on 254 samples, validate on 110 samples
Epoch 1/10
254/254 [==============================] - 7s 27ms/step - loss: 2.0723 - accuracy: 0.1496 - val_loss: 2.2019 - val_accuracy: 0.0000e+00
Epoch 2/10
254/254 [==============================] - 6s 25ms/step - loss: 2.0286 - accuracy: 0.1969 - val_loss: 2.2238 - val_accuracy: 0.0000e+00
Epoch 3/10
254/254 [==============================] - 6s 25ms/step - loss: 2.0188 - accuracy: 0.2008 - val_loss: 2.2229 - val_accuracy: 0.0000e+00
Epoch 4/10
254/254 [==============================] - 6s 26ms/step - loss: 1.9946 - accuracy: 0.2756 - val_loss: 2.2292 - val_accuracy: 0.0000e+00
Epoch 5/10
254/254 [==============================] - 7s 26ms/step - loss: 1.9761 - accuracy: 0.2835 - val_loss: 2.3782 - val_accuracy: 0.0000e+00
Epoch 6/10
254/254 [==============================] - 6s 25ms/step - loss: 1.9041 - accuracy: 0.3110 - val_loss: 2.2966 - val_accuracy: 0.0091
Epoch 7/10
254/254 [==============================] - 7s 26ms/step - loss: 1

In [ ]:
img_name

In [ ]:
predictions = model.predict(x_test)
predictions = np.argmax(predictions,axis = 1)
y_maps = dict()
y_maps = {v:k for k,v in class_names_label.items()}
print(y_maps)
print(predictions)
pred_labels = [y_maps[k] for k in predictions]
print(pred_labels)
sub = pd.DataFrame({'Image':img_name,'Target':pred_labels})
sub.to_csv('Solution.csv', index = False)